Mémoire de Master 2 Humanités Numériques et Master 2 Sociologie d'Anne-Lou Pichet. 
"Être Lecteur sur le Réseau Instagram : 
Sociologie et humanités numériques, regards croisés sur la pratique de la lecture sur Instagram. "


Ce script fait parti des annexes  de mon mémoire de recherche," Être Lecteur sur le Réseau Instagram : 
Sociologie et humanités numériques, regards croisés sur la pratique de la lecture sur Instagram." réalisé dans le cadre du Master Humanités numériques et du Master Sociologie (Analyse des sociétés contemporaines) à l’Université Lumière Lyon 2, durant l’année universitaire 2024-2025.


Mon mémoire cherche à comprendre comment se structure et évolue l’engagement dans la pratique de la lecture sur Instagram de personnes animant des comptes Instagram parlant de livres et lectures. Pour cela, nous avons mené des entretiens sociologiques avec cette population, et avons analysé leurs posts Instagram, à l'aide d'indicateurs statistiques et de méthodes de clustering. 
Au sein de ce script Python, je construis les indicateurs statistiques utilisés lors des analyses et mobilisés lors des clusterings. 

# Chargement des bibliothèques et fonctions. 

In [31]:
import pandas as pd
import numpy as np
import re
import spacy
#!python -m spacy download fr_core_news_lg

nlp = spacy.load("fr_core_news_lg")

## Fonctions

In [32]:
# Listes de noms par type de carrière
carriere_loisir_socio = ['Agathe','Alya', 'Jules', 'Salomé', 'Alice ']
carriere_semi_professionnelle = ['Clément', 'Claire ', 'Amélie', 'Capucine', 'Hélène', 'Christine ', 'Laureline', 'Elisabeth', 'Jade']
carriere_milieu_socio = ['Aline', 'Arthur', 'Emma', 'Ariane', 'Maude']
# Fonction pour déterminer le type de carrière
def get_type_carriere(nom):
    if nom in carriere_loisir_socio:
        return 'loisir'
    elif nom in carriere_semi_professionnelle:
        return 'semi_professionnelle'
    elif nom in carriere_milieu_socio:
        return 'milieu'
    else:
        return 'inconnu'

In [33]:
# Fonction pour extraire et classifier les hashtags
def analyser_hashtags(post):
    hashtags = re.findall(r"#(\w+)", str(post))
    total_hashtags = len(hashtags)
    return {
        "Contient_Hashtags": total_hashtags > 0,
        "Nombre_Hashtags": total_hashtags,
        "Liste_Hashtags": hashtags
    }


In [34]:
# Fonction pour extraire et classifier les hashtags dans les posts (commençant par un #)
def analyser_hashtags(post):
    hashtags = re.findall(r"#(\w+)", str(post))
    total_hashtags = len(hashtags)
    return {
        "Contient_Hashtags": total_hashtags > 0,
        "Nombre_Hashtags": total_hashtags,  
        "Liste_Hashtags": hashtags,  
    }

In [35]:
# Fonction pour extraire et classifier les mentions dans les posts (commençant par un @)
def analyser_mentions(post):
    mentions = re.findall(r"@(\w+)", str(post))
    total_mentions = len(mentions)
    mentions_auteurs = [m for m in mentions if m.startswith("A")]  # Mentions d'auteurs
    mentions_maisons = [m for m in mentions if m.startswith("ME")]  # Mentions de maisons d'édition
    mentions_pairs = [m for m in mentions if m.startswith("Pa")]  # Mentions de pairs
    mentions_autres = [m for m in mentions if not (m.startswith("A") or m.startswith("ME") or m.startswith("Pa"))]  # Mentions autres
    return {
        "Contient_Mentions": total_mentions > 0,  
        "Nombre_Mentions": total_mentions,  
        "Mentions_Auteurs": len(mentions_auteurs),  
        "Mentions_Maisons": len(mentions_maisons),  
        "Mentions_Autres": len(mentions_autres),  
        "Mentions_Pairs": len(mentions_pairs),  
        "Contient_Mentions_Pairs": len(mentions_pairs) > 0,  
        "Contient_Mentions_Auteurs": len(mentions_auteurs) > 0,
        "Contient_Mentions_Autres": len(mentions_autres) > 0,
        "Contient_Mentions_Maisons": len(mentions_maisons) > 0
    }

# Chargement des données et prétraitements de celles-ci  

Ce code a pour but de transformer la structure de mes données pour pouvoir, par la suite, construire des statistiques à leur propos.

In [36]:
data_brutes = pd.read_excel("Posts_Anon.xlsx") # fichier d'origine des données à transformer 
images_brutes = pd.read_excel("Images_Posts.xlsx", engine='openpyxl') # fichier contenant les données des images

### Transformation des données brutes

In [37]:
# Liste pour stocker les données transformées
format_propre = []

# Pour chaque auteur dans le DataFrame, on parcourt les colonnes et on remet en forme les données
for _, row in data_brutes.iterrows():
    for col in data_brutes.columns:
        match = re.match(r"Contenu_([A-Za-z0-9]+)_Post", col)  # On extrait le code du post (ex: F1, L2, A1E3...) 
        if match:  
            code_post = match.group(1)  # On récupère le code du post
            
            # Récupérer les valeurs associées aux autres attributs
            contenu = row[f"Contenu_{code_post}_Post"]
            type_post = row.get(f"Type_{code_post}_Post", None)
            annexes = row.get(f"Annexes_{code_post}_Post", None)
            date_post = row.get(f"Date_{code_post}_Post", None)
            
            code_propre = None
            nombre_propre = None
            code_match = re.match(r"^(.+?)(?=\d+$)", code_post)  # Tout sauf les derniers chiffres
            nombre_match = re.search(r"(\d+)$", code_post)       # Les derniers chiffres uniquement
            code_propre = code_match.group(1)
            nombre_propre = nombre_match.group(1)
            nombre_propre = float(nombre_propre)

            # Si un contenu ou une date existe, ajouter une nouvelle ligne
            if pd.notna(date_post) or pd.notna(contenu):
                format_propre.append({
                    "Nom_d_emprunt": row["Nom d'emprunt"],
                    "Contenu_Post": contenu,
                    "Type_Post": type_post,
                    "Code_Post": code_post, 
                    "Code": code_propre,
                    "Nombre": nombre_propre,
                    "Annexes_Post": annexes,
                    "Date_Post": date_post
                })
             
# Créer un DataFrame final
data_propre = pd.DataFrame(format_propre)


### Nettoyage Posts

In [38]:
# Dictionnaire de correspondance pour corriger les erreurs
corrections = {
    "Vidéos": "Vidéo",
    "Vdéo": "Vidéo",
    "Videos": "Vidéo",
    "DIvers": "Divers",
    "Dievrs": "Divers",
    "Dvers": "Divers",
    "Recap": "Récap",
    "chronique": "Chronique",
    "recommandations": "Recommandations",
    "Recommendations": "Recommandations",
    "Auteur": "Professionnel",  # Si "Auteur" désigne un post pro
    "Extrait": "Extrait",
    "Citation": "Citation",
    "Podcast": "Audio",
    "Haul": "Haul",
    "Challenge": "Challenge",
    "Présentation": "Présentation",
    "Bilan": "Bilan",
    "Récap": "Récap",
}

# Supprimer les valeurs clairement incorrectes
valeurs_a_supprimer = {'', 'p', ']', 'www.instagram.com', 'https:', 'CthTGTTIj7u'}

# Fonction pour nettoyer les types de post
def clean_type_post(t):
    if pd.isna(t):
        return None
    cleaned_values = {corrections.get(x.strip(), x.strip()) for x in t.split('/')}
    cleaned_values.difference_update(valeurs_a_supprimer)  
    cleaned_values = {re.sub(r'\[.*?\]|https?:\/\/\S+', '', x) for x in cleaned_values}
    cleaned_values.discard('')
    return '/'.join(sorted(cleaned_values)) if cleaned_values else None  

# Appliquer le nettoyage
data_propre['Type_Post'] = data_propre['Type_Post'].apply(clean_type_post)

# Afficher les nouvelles valeurs uniques
print(data_propre['Type_Post'].unique())

['Chronique' 'Recommandations/Vidéo' 'Chronique/Recommandation'
 'Récap/Vidéo' 'Présentation' 'Divers' 'Recommandations' 'Chronique/Récap'
 'Avent/Chronique' 'Chronique/Vidéo' 'Haul/Vidéo' 'Divers/Vidéo'
 'Divers/Recommandations' 'Citation'
 'Divers [https:/Recommandations/Vidéo' 'Interview' 'Chronique/Divers'
 'Chronique/Recommandations' 'Chronique/Recommandations/Récap' 'Haul'
 'Audio/Interview' 'Audio/Divers/Vidéo' 'Audio/Divers/Interview'
 'Divers/Recommandations/Vidéo' 'Divers/Professionnel' 'Récap' None
 'Chronique/Professionnel' 'Chronique/Citation'
 'Recommandations/Récap/Vidéo' 'Audio/Chronique' 'Chronique/Haul'
 'Audio/Extrait' 'Chronique/Divers/Vidéo' 'Divers/Haul/Vidéo' 'Bilan'
 'Challenge' 'Chronique/Récap/Vidéo' 'Citation/Divers']


### Déterminer la nature des posts

In [39]:
def determine_post_type(row):
    type_post = row['Type_Post']
    
    if pd.isna(type_post):
        return None  # Aucun type si NaN
    
    types = set(type_post.split('/'))  # Séparer les types multiples

    # On garde toutes les catégories valides
    categories = set()
    if 'Vidéo' in types:
        categories.add('Vidéo')
    if 'Audio' in types:
        categories.add('Audio')
    if pd.notna(row['Contenu_Post']):
        categories.add('Écrit')

    return '/'.join(sorted(categories)) if categories else None  # Trier pour homogénéité

# Appliquer la fonction mise à jour
data_propre['Type_Post'] = data_propre.apply(determine_post_type, axis=1)

# Créer les colonnes binaires
data_propre['N_Video'] = data_propre['Type_Post'].str.contains('Vidéo', na=False).astype(int)
data_propre['N_Audio'] = data_propre['Type_Post'].str.contains('Audio', na=False).astype(int)
data_propre['N_Ecrit'] = data_propre['Type_Post'].str.contains('Écrit', na=False).astype(int)

data_propre.drop(columns=['Type_Post'], inplace=True)

### Merge avec les données des images

In [40]:
# ajout d'un index commun
data_propre['index'] = data_propre.groupby(['Nom_d_emprunt', 'Date_Post']).cumcount()
images_brutes['index'] = images_brutes.groupby(['Nom_d_emprunt', 'Date_Post']).cumcount()

# conversion des colonnes spécifiques en numérique
images_brutes[['Plusieurs_Images', 'Personne', 'PostProduction', 'Mots']] = images_brutes[['Plusieurs_Images', 'Personne', 'PostProduction', 'Mots']].apply(pd.to_numeric, errors='coerce')

# Fusion des données
base_complete = pd.merge(data_propre, images_brutes, on=['Nom_d_emprunt', 'Date_Post', 'index'], how='left')


# Save the merged_data to an Excel file
#base_complete.to_excel("Base_Posts_Anon.xlsx", index=False)

## Données traitées pour les statistiques

In [41]:
data = pd.read_excel("Base_Posts_Anon.xlsx")

## Gestion des périodes et des identifiants.

In [42]:
# On tranforme les périodes pour qu'elles soient plus lisibles
data["Date_Post"] = pd.to_datetime(data["Date_Post"], errors="coerce")
data["Periode"] = data["Code"].str.extract(r"^([A-Za-z]+[0-9]*)") 

recodage_periodes = {"F": 1, "A1": 2, "A2": 3, "L": 4} # Dictionnaire de recodage des périodes
# On gére les posts épinglés
data["Periode"] = data["Periode"].apply(lambda x: recodage_periodes.get(x, "E"))

# On ajoute un Identifiant pour chaque post
data["ID_Post"] = range(1, len(data) + 1)
data.set_index("ID_Post", inplace=True)

## Ajout des types de carrières 

In [43]:
data['type_carriere'] = data['Nom_d_emprunt'].apply(get_type_carriere)

## Réattribuer les posts épinglés

In [44]:
data["Date_Post"] = pd.to_datetime(data["Date_Post"], errors="coerce")
data["Période_Bis"] = data["Periode"] 
data = data.sort_values(by=["Nom_d_emprunt", "Periode", "Date_Post"])

In [45]:
# Réaffectation des posts "E"
for auteur in data["Nom_d_emprunt"].unique():
    auteur_data = data[data["Nom_d_emprunt"] == auteur].copy()

    posts_E = auteur_data[auteur_data["Periode"] == "E"]
    autres_periodes = auteur_data[auteur_data["Periode"] != "E"]

    for periode in autres_periodes["Periode"].unique():
        periode_data = autres_periodes[autres_periodes["Periode"] == periode]
        if periode_data["Date_Post"].isnull().all():
            continue
        min_date = periode_data["Date_Post"].min()
        max_date = periode_data["Date_Post"].max()

        for idx_E, row_E in posts_E.iterrows():
            date_E = row_E["Date_Post"]
            if pd.notnull(date_E) and min_date <= date_E <= max_date:
                data.at[idx_E, "Période_Bis"] = periode

In [46]:
# Création du tableau des plages par période
plages_periode = []

for auteur in data["Nom_d_emprunt"].unique():
    auteur_data = data[(data["Nom_d_emprunt"] == auteur) & (data["Periode"] != "E")]

    for periode in auteur_data["Periode"].unique():
        periode_data = auteur_data[auteur_data["Periode"] == periode]
        if periode_data["Date_Post"].isnull().all():
            continue

        plages_periode.append({
            "Auteur": auteur,
            "Periode": periode,
            "Date_Début": periode_data["Date_Post"].min(),
            "Date_Fin": periode_data["Date_Post"].max()
        })

plages = pd.DataFrame(plages_periode)

In [47]:
# Vérification des rattachements des posts épinglés ("E")
posts_E_originaux = data[data["Periode"] == "E"]

verifications = []

for idx, row in posts_E_originaux.iterrows():
    auteur = row["Nom_d_emprunt"]
    date_post = row["Date_Post"]
    periode_bis = row["Période_Bis"]

    valide = False

    auteur_data = data[(data["Nom_d_emprunt"] == auteur) & (data["Periode"] != "E")]
    for periode in auteur_data["Periode"].unique():
        periode_data = auteur_data[auteur_data["Periode"] == periode]
        if periode_data["Date_Post"].isnull().all():
            continue
        min_date = periode_data["Date_Post"].min()
        max_date = periode_data["Date_Post"].max()

        if pd.notnull(date_post) and min_date <= date_post <= max_date:
            valide = (periode_bis == periode)
            break

    verifications.append({
        "Index": idx,
        "Auteur": auteur,
        "Date_Post": date_post,
        "Période_Bis": periode_bis,
        "Date_dans_une_plage_connue": valide
    })

verifications = pd.DataFrame(verifications)

In [48]:
# On vérifie les dates des posts épinglés par rapport aux plages connues
verifs = verifications.merge(plages, how="left",
                                  left_on=["Auteur", "Période_Bis"],
                                  right_on=["Auteur", "Periode"])

verifs = verifs[[
    "Index", "Auteur", "Date_Post", "Période_Bis", "Date_dans_une_plage_connue",
    "Date_Début", "Date_Fin"
]].sort_values(by=["Auteur", "Date_Post"])


print("Nombre de posts 'E' au départ :", len(posts_E_originaux))
print("Nombre de posts 'E' encore non rattachés :", (data["Période_Bis"] == "E").sum())
print("\nRattachements incorrects ou non attribués :")
verifs[verifs["Date_dans_une_plage_connue"] == False] # Pour voir si il y a des erreurs

Nombre de posts 'E' au départ : 22
Nombre de posts 'E' encore non rattachés : 17

Rattachements incorrects ou non attribués :


,Index,Auteur,Date_Post,Période_Bis,Date_dans_une_plage_connue,Date_Début,Date_Fin
0,133,Amélie,2023-08-11,E,False,NaT,NaT
1,134,Amélie,2024-02-22,E,False,NaT,NaT
2,225,Capucine,2022-06-03,E,False,NaT,NaT
3,226,Capucine,2022-11-14,E,False,NaT,NaT
4,227,Capucine,2023-02-16,E,False,NaT,NaT
5,268,Christine,2023-08-24,E,False,NaT,NaT
6,299,Claire,2024-09-06,E,False,NaT,NaT
7,300,Claire,2024-09-06,E,False,NaT,NaT
8,301,Claire,2024-10-01,E,False,NaT,NaT
11,404,Emma,2024-03-31,E,False,NaT,NaT


In [49]:
# Vérification des périodes pour certains enquêtés pour vérifier les rattachements à la main 
enquetes = ['Amélie', 'Capucine', 'Christine', 'Claire', 'Emma', 'Jade', 'Laureline', 'Salomé']

for auteur in data["Nom_d_emprunt"].unique():
    if auteur in enquetes:
        auteur_data = data[(data["Nom_d_emprunt"] == auteur) & (data["Periode"] != "E")]
        print(f"\nPériodes pour l'auteur {auteur} :")
        for periode in auteur_data["Periode"].unique():
            periode_data = auteur_data[auteur_data["Periode"] == periode]
            if not periode_data["Date_Post"].isnull().all():
                min_date = periode_data["Date_Post"].min()
                max_date = periode_data["Date_Post"].max()
                print(f"Période {periode}: Début: {min_date}, Fin: {max_date}")


Périodes pour l'auteur Amélie :
Période 1: Début: 2023-06-11 00:00:00, Fin: 2023-06-23 00:00:00
Période 2: Début: 2023-12-27 00:00:00, Fin: 2024-01-03 00:00:00
Période 4: Début: 2024-10-06 00:00:00, Fin: 2024-10-16 00:00:00

Périodes pour l'auteur Capucine :
Période 1: Début: 2017-06-05 00:00:00, Fin: 2019-08-21 00:00:00
Période 2: Début: 2024-04-01 00:00:00, Fin: 2024-06-18 00:00:00
Période 4: Début: 2024-05-27 00:00:00, Fin: 2024-10-09 00:00:00

Périodes pour l'auteur Emma :
Période 1: Début: 2020-08-19 00:00:00, Fin: 2020-09-09 00:00:00
Période 2: Début: 2023-12-30 00:00:00, Fin: 2024-01-28 00:00:00
Période 3: Début: 2024-08-24 00:00:00, Fin: 2024-09-30 00:00:00
Période 4: Début: 2024-09-01 00:00:00, Fin: 2024-10-13 00:00:00

Périodes pour l'auteur Jade :
Période 1: Début: 2022-08-06 00:00:00, Fin: 2022-08-22 00:00:00
Période 2: Début: 2023-11-19 00:00:00, Fin: 2023-12-13 00:00:00
Période 3: Début: 2024-10-10 00:00:00, Fin: 2024-11-03 00:00:00
Période 4: Début: 2024-09-25 00:00:00,

In [50]:
data.to_excel("Base_Posts_Anon.xlsx") # Pour sauvegarder les modifications la première fois que le script est exécuté

# Création des indicateurs statistiques

## Médiane des posts

In [80]:
# obtenir le nombre total et la médiane de posts par enquêté
posts_par_enquete = data.groupby('Nom_d_emprunt').size().reset_index(name='Nombre de Posts')
mediane_posts = posts_par_enquete['Nombre de Posts'].median()
mediane_posts 

31.0

## Fréquence de posts 

On analyse ici la fréquence à laquelle nos enquêtés postent sur Instagram à propos de leurs lectures. 
Pour cela, nous calculons la fréquence entre chaque posts d'une même période et pour un même enquêté. Nous croiserons par la suite les résulats avec plusieurs autres indicateurs.  

In [81]:
data_freq = data.sort_values(by=["Nom_d_emprunt", "Période_Bis", "Date_Post"])

frequence = []

for auteur in data_freq["Nom_d_emprunt"].unique():
    auteur_data = data_freq[data_freq["Nom_d_emprunt"] == auteur]
    carriere = auteur_data["type_carriere"].unique()
    for periode in auteur_data["Période_Bis"].unique():
        periode_data = auteur_data[auteur_data["Période_Bis"] == periode]
        
        if periode_data["Date_Post"].isnull().all():
            frequence.append({
                "Nom_d_emprunt": auteur,
                "Période_Bis": periode,
                "Carrière": carriere,
                "Nombre_de_Posts": 0,
                "Intervalle_Moyen": None,
                "Intervalle_Minimum": None,
                "Intervalle_Maximum": None,
                "Ecart_Type_Intervalle": None
            })
            continue
        
        nombre_de_posts = len(periode_data)
        
        periode_data = periode_data.sort_values(by="Date_Post")
        periode_data["Intervalle_Posts"] = periode_data["Date_Post"].diff().dt.days
        
        periode_data = periode_data.dropna(subset=["Intervalle_Posts"])
        
        if not periode_data.empty:
            frequence.append({
                "Nom_d_emprunt": auteur,
                "Période_Bis": periode,
                "Carrière": carriere,
                "Nombre_de_Posts": nombre_de_posts, 
                "Intervalle_Moyen": periode_data["Intervalle_Posts"].mean(),
                "Intervalle_Minimum": periode_data["Intervalle_Posts"].min(),
                "Intervalle_Maximum": periode_data["Intervalle_Posts"].max(),
                "Intervalle_Median": periode_data["Intervalle_Posts"].median(),
                "Ecart_Type_Intervalle": periode_data["Intervalle_Posts"].std()
            })
        else:
            # Si les intervalles ne peuvent pas être calculés, on ajoute des valeurs par défaut
            frequence.append({
                "Nom_d_emprunt": auteur,
                "Période_Bis": periode,
                "Carrière": carriere,
                "Nombre_de_Posts": nombre_de_posts,
                "Intervalle_Moyen": None,
                "Intervalle_Minimum": None,
                "Intervalle_Maximum": None,
                "Intervalle_Mediane": None,
                "Ecart_Type_Intervalle": None
            })

frequence = pd.DataFrame(frequence)
frequence.rename(columns={"Période_Bis": "Periode",}, inplace=True)
frequence.head()



,Nom_d_emprunt,Periode,Carrière,Nombre_de_Posts,Intervalle_Moyen,Intervalle_Minimum,Intervalle_Maximum,Intervalle_Median,Ecart_Type_Intervalle,Intervalle_Mediane
0,Agathe,1,[loisir],2,23.000000,23.0,23.0,23.0,NaN,NaN
1,Alice,1,[loisir],10,1.666667,0.0,3.0,2.0,0.866025,NaN
2,Alice,2,[loisir],10,7.666667,2.0,23.0,5.0,6.538348,NaN
3,Alice,4,[loisir],10,12.444444,2.0,23.0,12.0,7.763876,NaN
4,Aline,1,[milieu],10,8.333333,0.0,68.0,1.0,22.377444,NaN


### Analyse des fréquences selon les carrières

On croise l'analyse des fréquences de posts avec les carrières. 
Comme nous avons un outlier impactant fortement les données des fréquences (Capucine), nous avons voulu voir la différence en l'enlevant. 

In [82]:
freq_carriere = []

frequence["Carrière"] =frequence["Carrière"].apply(lambda x: str(x))
for carriere in frequence["Carrière"].unique():
    carriere_data = frequence[frequence["Carrière"] == carriere]

    # Suppression de l'outlier pour voir son effet sur la moyenne
    max_val = carriere_data['Intervalle_Moyen'].max()
    carriere_data_sans_outlier = carriere_data[carriere_data['Intervalle_Moyen'] != max_val]

    freq_carriere.append({
        "Carrière": carriere,
        "Nombre de posts": carriere_data['Nombre_de_Posts'].sum(),
        "Intervalle minimum": carriere_data['Intervalle_Minimum'].min(),
        "Intervalle maximum": carriere_data['Intervalle_Maximum'].max(),
        "Intervalle moyen": carriere_data['Intervalle_Moyen'].mean(),
        "Intervalle médian": carriere_data['Intervalle_Median'].median(),
        "Intervalle moyen (sans outlier)": carriere_data_sans_outlier['Intervalle_Moyen'].mean()
    })

freq_carriere = pd.DataFrame(freq_carriere)
freq_carriere

,Carrière,Nombre de posts,Intervalle minimum,Intervalle maximum,Intervalle moyen,Intervalle médian,Intervalle moyen (sans outlier)
0,['loisir'],117,0.0,79.0,9.630769,4.0,8.516667
1,['milieu'],173,0.0,68.0,9.080864,5.0,8.490850
2,['semi_professionnelle'],306,0.0,640.0,23.171569,2.0,17.964646


Nous allons maintenant chercher les statistiques de fréquences croisées avec les carrières et les périodes. 

In [83]:
frequence.columns

Index(['Nom_d_emprunt', 'Periode', 'Carrière', 'Nombre_de_Posts',
       'Intervalle_Moyen', 'Intervalle_Minimum', 'Intervalle_Maximum',
       'Intervalle_Median', 'Ecart_Type_Intervalle', 'Intervalle_Mediane'],
      dtype='object')

In [84]:
freq_carriere_periode = []
for carriere in frequence["Carrière"].unique():
    carriere_data = frequence[frequence["Carrière"] == carriere]

    # Boucle sur chaque période sauf les posts épinglés ('E')
    for periode in carriere_data["Periode"].unique():
        if periode == 'E':
            continue
        periode_data = carriere_data[carriere_data["Periode"] == periode]

        nb_posts = periode_data["Nombre_de_Posts"].sum()
        intervalle_min = periode_data["Intervalle_Minimum"].min()
        intervalle_max = periode_data["Intervalle_Maximum"].max()
        intervalle_moy = periode_data["Intervalle_Moyen"].mean()
        intervalle_med = periode_data["Intervalle_Median"].median()

        freq_carriere_periode.append({
            "Carrière": carriere,
            "Période": periode,
            "Nombre de posts": nb_posts,
            "Intervalle minimum": intervalle_min,
            "Intervalle maximum": intervalle_max,
            "Intervalle moyen": round(intervalle_moy, 2), 
            "Intervalle médian": round(intervalle_med, 2),
        })

freq_carriere_periode = pd.DataFrame(freq_carriere_periode)

freq_carriere_periode.sort_values(by=["Carrière", "Période"])

,Carrière,Période,Nombre de posts,Intervalle minimum,Intervalle maximum,Intervalle moyen,Intervalle médian
0,['loisir'],1,43,0.0,23.0,6.30,2.00
1,['loisir'],2,40,0.0,79.0,12.28,4.50
2,['loisir'],4,33,0.0,79.0,11.15,8.25
3,['milieu'],1,50,0.0,68.0,7.13,2.00
4,['milieu'],2,51,0.0,33.0,7.67,5.00
5,['milieu'],3,20,1.0,45.0,10.22,9.50
6,['milieu'],4,50,1.0,45.0,11.00,10.00
7,['semi_professionnelle'],1,90,0.0,640.0,21.70,2.00
8,['semi_professionnelle'],2,91,0.0,14.0,3.16,3.00
10,['semi_professionnelle'],3,20,1.0,5.0,2.28,2.00


### Selon les périodes

On va ici croiser les statistiques de fréquence avec les périodes, pour identifier de potentielles variations dans le temps. Pour cela, on inclut des pondérations car tout les enquêtés n'ont pas le meme nombre de posts pour chaque période. 

In [85]:
frequence

,Nom_d_emprunt,Periode,Carrière,Nombre_de_Posts,Intervalle_Moyen,Intervalle_Minimum,Intervalle_Maximum,Intervalle_Median,Ecart_Type_Intervalle,Intervalle_Mediane
0,Agathe,1,['loisir'],2,23.000000,23.0,23.0,23.0,NaN,NaN
1,Alice,1,['loisir'],10,1.666667,0.0,3.0,2.0,0.866025,NaN
2,Alice,2,['loisir'],10,7.666667,2.0,23.0,5.0,6.538348,NaN
3,Alice,4,['loisir'],10,12.444444,2.0,23.0,12.0,7.763876,NaN
4,Aline,1,['milieu'],10,8.333333,0.0,68.0,1.0,22.377444,NaN
...,...,...,...,...,...,...,...,...,...,...
62,Maude,4,['milieu'],10,19.111111,8.0,34.0,22.0,9.225568,NaN
63,Salomé,1,['loisir'],11,4.700000,0.0,9.0,4.0,2.790858,NaN
64,Salomé,2,['loisir'],10,5.777778,1.0,17.0,4.0,5.517648,NaN
65,Salomé,4,['loisir'],10,4.000000,1.0,10.0,3.0,3.278719,NaN


In [86]:
frequence_periode = pd.DataFrame(columns=["Période"])
for periode in frequence["Periode"].unique():
    periode_data = frequence[frequence["Periode"] == periode]
    # intervalle moyen
    intervalle_moyen_pondere = (periode_data["Intervalle_Moyen"] * periode_data["Nombre_de_Posts"]).sum() / periode_data["Nombre_de_Posts"].sum()
    # Intervalle minimum
    intervalle_minimum = periode_data["Intervalle_Minimum"].min()
    # Intervalle maximum
    intervalle_maximum = periode_data["Intervalle_Maximum"].max()
    # Intervalle median
    mediane_intervalle = periode_data["Intervalle_Moyen"].median()
    #ecart type pondéré
    ecart_type_pondere = (periode_data["Ecart_Type_Intervalle"] * periode_data["Nombre_de_Posts"]).sum() / periode_data["Nombre_de_Posts"].sum()
    # Nombre total de posts dans cette période
    nombre_posts = periode_data["Nombre_de_Posts"].sum()

    # Ajouter les résultats au tableau récapitulatif
    nouvelle_ligne = pd.DataFrame([{
        "Période": periode,
        "Intervalle_Moyen": intervalle_moyen_pondere,
        "Médiane_Intervalle": mediane_intervalle,
        "Intervalle_Minimum": intervalle_minimum,
        "Intervalle_Maximum": intervalle_maximum,
        "Ecart_Type": ecart_type_pondere,
        "Intervalle_Mediane": mediane_intervalle,
        "Nombre_de_Posts": nombre_posts,

        }])


    frequence_periode = pd.concat([frequence_periode, nouvelle_ligne], ignore_index=True)

# Ajout des données pour toutes les périodes 

ligne_total = pd.DataFrame([{
    "Période": "Total",
    "Intervalle_Moyen": frequence_periode["Intervalle_Moyen"].mean(),
    "Médiane_Intervalle": frequence_periode["Médiane_Intervalle"].median(),
    "Intervalle_Minimum": frequence_periode["Intervalle_Minimum"].min(),
    "Intervalle_Maximum": frequence_periode["Intervalle_Maximum"].max(),
    "Ecart_Type": frequence_periode["Ecart_Type"].mean(),
    "Nombre_de_Posts": frequence_periode["Nombre_de_Posts"].sum(),
}])

frequence_periode = pd.concat([frequence_periode, ligne_total], ignore_index=True)

frequence_periode


,Période,Intervalle_Moyen,Médiane_Intervalle,Intervalle_Minimum,Intervalle_Maximum,Ecart_Type,Intervalle_Mediane,Nombre_de_Posts
0,1,13.363267,2.333333,0.0,640.0,19.208315,2.333333,183.0
1,2,6.411722,5.000000,0.0,79.0,5.526431,5.000000,182.0
2,4,7.477458,6.666667,0.0,79.0,5.735821,6.666667,174.0
3,3,6.250000,3.388889,1.0,45.0,4.831480,3.388889,40.0
4,E,81.470588,71.500000,0.0,242.0,28.949784,71.500000,17.0
5,Total,22.994607,5.000000,0.0,640.0,12.850366,NaN,596.0


## Longueur des posts et des annexes 

On va ici analyser le contenu des posts par leur longeur (à la fois le corps des posts mais aussi leurs éventuelles annexes). 

Nous avons pour cette analyse, tenté plusieurs choses : 
- un calcul de la longeur en mots et en caractères. On a finalement retenu ce dernier indicateurs pour nos analyses. 
- nous avons envisagé de faire une analyse plus poussée sur le langage utilisée dans les posts de nos enquêtés, avant de plutôt nous tourner vers des clusterings pour identifier des clusters de pratiques et d'enquêtés. Dans un travail suivant, il serait alors pertinent de poursuivre par une analyse TAL. 

In [87]:
# Calculer la longueur des posts (en caractères et mots)
data["Contenu_Post"] = data["Contenu_Post"].astype(str)  # Gérer les NaN
data["Longueur_Mots"] = data["Contenu_Post"].apply(lambda x: len(x.split()))  # Longueur en mots
data["Longueur_Contenu"] = data["Contenu_Post"].apply(lambda x: len(str(x)) if pd.notna(x) else 0)

data["Presence_Annexes"] = data["Annexes_Post"].apply(lambda x: bool(pd.notna(x) and str(x).strip() and str(x).lower() != 'nan'))
# On identifie les annexes dans les posts avec [AP]
data["Annexes_Post"] = data["Annexes_Post"].astype(str)  # Convertir en chaînes de caractères
data["Longueur_Annexes"] = data["Annexes_Post"].str.replace(r"\[AP\]", "", regex=True).apply(len)
data["Nombre_Annexes"] = data["Annexes_Post"].apply(
    lambda x: len(str(x).split("[AP]")) if pd.notna(x) and str(x).strip() and str(x).lower() != 'nan' else 0
)

# Calculer la longueur totale (contenu + annexes)
data["Longueur_Totale"] = data["Longueur_Contenu"] + data["Longueur_Annexes"]


# Colonnes pour les caractéristiques NLP des posts et de leurs annexes
data["Entites_Nommees"] = None
data["Tags_POS"] = None
data["Vecteur_Norm"] = None
data["Entites_Nommees_Annexes"] = None
data["Tags_POS_Annexes"] = None
data["Vecteur_Norm_Annexes"] = None

# Analyse NLP sur chaque post et ses annexes
for i, row in data.iterrows():
    doc_post = nlp(row["Contenu_Post"])
    entites_post = [(ent.text, ent.label_) for ent in doc_post.ents]
    pos_tags_post = [(token.text, token.pos_) for token in doc_post]
    vecteur_norm_post = doc_post.vector_norm
       
    # On ajoute les résultats pour le corps du post
    data.at[i, "Entites_Nommees"] = str(entites_post)
    data.at[i, "Tags_POS"] = str(pos_tags_post)
    data.at[i, "Vecteur_Norm"] = vecteur_norm_post
    
    # Puis on fait de même pour les annexes
    annexes = row["Annexes_Post"].split("[AP]") if row["Annexes_Post"].strip() else []
    entites_annexes = []
    pos_tags_annexes = []
    vecteur_norm_annexes = 0
    
    if annexes:
        for annexe in annexes:
            annexe = annexe.strip()
            if annexe and annexe != "nan": 
                doc_annexe = nlp(annexe)
                entites_annexes.extend([(ent.text, ent.label_) for ent in doc_annexe.ents])
                pos_tags_annexes.extend([(token.text, token.pos_) for token in doc_annexe])
                vecteur_norm_annexes += doc_annexe.vector_norm 
    data.at[i, "Entites_Nommees_Annexes"] = str(entites_annexes)
    data.at[i, "Tags_POS_Annexes"] = str(pos_tags_annexes)
    data.at[i, "Vecteur_Norm_Annexes"] = vecteur_norm_annexes


### Analyse de la longeur des posts

In [88]:
# Longueur totale (contenu + annexes)
longueur_totale_data = data.dropna(subset=["Longueur_Totale"])

#rajouter la précense des annexes en pourcentages
stats_globales_longueur = {
    "Type": "Global",
    "Nombre_de_Posts": len(longueur_totale_data),
    "Longueur_Moyenne": longueur_totale_data["Longueur_Totale"].mean(),
    "Longueur_Minimale": longueur_totale_data["Longueur_Totale"].min(),
    "Longueur_Maximale": longueur_totale_data["Longueur_Totale"].max(),
    "Longueur_Mediane": longueur_totale_data["Longueur_Totale"].median(),
    "Ecart_Type_Longueur": longueur_totale_data["Longueur_Totale"].std(),
    "Pourcentage_Annexes": (longueur_totale_data["Presence_Annexes"].sum() / len(longueur_totale_data)) * 100,
    "Nombres d'Annexes" : longueur_totale_data["Nombre_Annexes"].sum()
}

In [89]:
periode_longueur = []

for periode in data["Periode"].dropna().unique():
    periode_data = data[data["Periode"] == periode].dropna(subset=["Longueur_Totale"])
    
    if not periode_data.empty:
        periode_longueur.append({
            "Type": f"Période: {periode}",
            "Nombre_de_Posts": len(periode_data),
            "Longueur_Moyenne": periode_data["Longueur_Totale"].mean(),
            "Longueur_Minimale": periode_data["Longueur_Totale"].min(),
            "Longueur_Maximale": periode_data["Longueur_Totale"].max(),
            "Longueur_Mediane": periode_data["Longueur_Totale"].median(),
            "Ecart_Type_Longueur": periode_data["Longueur_Totale"].std(),
            "Pourcentage_Annexes": (periode_data["Presence_Annexes"].sum() / len(periode_data)) * 100, 
            "Nombres d'Annexes" : periode_data["Nombre_Annexes"].sum()  

        })

In [90]:
# On affiche les résultats globaux et par période
longueur = pd.DataFrame([stats_globales_longueur] + periode_longueur)

longueur.head()

,Type,Nombre_de_Posts,Longueur_Moyenne,Longueur_Minimale,Longueur_Maximale,Longueur_Mediane,Ecart_Type_Longueur,Pourcentage_Annexes,Nombres d'Annexes
0,Global,596,1383.709732,6,3778,1370.0,819.491358,13.926174,98
1,Période: 1,182,947.159341,6,3016,796.5,732.406570,9.340659,19
2,Période: 2,180,1574.000000,41,3232,1672.0,738.412051,14.444444,33
3,Période: 4,172,1538.180233,26,3778,1573.5,768.927861,15.116279,30
4,Période: 3,40,1775.925000,312,3497,1677.0,933.822181,25.000000,12


In [91]:

enquete_longueur = []

for (nom_enquete, periode), couple in data.groupby(["Nom_d_emprunt", "Periode"]):
    if not couple.empty:
        enquete_longueur.append({
            "Nom_d_emprunt": nom_enquete,
            "Periode": periode,
            "Nombre_de_Posts": len(couple),
            "Longueur_Moyenne": couple["Longueur_Totale"].mean(),
            "Longueur_Minimale": couple["Longueur_Totale"].min(),
            "Longueur_Maximale": couple["Longueur_Totale"].max(),
            "Ecart_Type_Longueur": couple["Longueur_Totale"].std(),
            "Longueur_Mediane": couple["Longueur_Totale"].median(),
            "Pourcentage_Annexes": (couple["Presence_Annexes"].sum() / len(couple)) * 100,
            "Nombres d'Annexes": couple["Nombre_Annexes"].sum()
        })
enquete_longueur = pd.DataFrame(enquete_longueur)
enquete_longueur = enquete_longueur.sort_values(by=["Nom_d_emprunt", "Periode"])
enquete_longueur

,Nom_d_emprunt,Periode,Nombre_de_Posts,Longueur_Moyenne,Longueur_Minimale,Longueur_Maximale,Ecart_Type_Longueur,Longueur_Mediane,Pourcentage_Annexes,Nombres d'Annexes
0,Agathe,1,2,1157.5,928,1387,324.562013,1157.5,0.0,0
1,Alice,1,10,878.6,127,1234,338.143691,933.0,0.0,0
2,Alice,2,10,1322.6,819,2030,406.776543,1256.0,0.0,0
3,Alice,4,10,1507.3,1212,2030,268.522728,1483.0,0.0,0
4,Aline,1,10,463.9,23,942,303.518423,544.5,0.0,0
...,...,...,...,...,...,...,...,...,...,...
64,Maude,4,10,1087.2,200,1961,550.957308,891.5,0.0,0
65,Salomé,1,10,1194.3,470,1540,349.222645,1302.5,0.0,0
66,Salomé,2,10,1547.6,290,2155,591.280757,1672.0,0.0,0
67,Salomé,4,10,1659.9,237,2341,607.292617,1788.0,10.0,1


In [92]:
longueur_carriere = []
for carriere in data["type_carriere"].unique():
    carriere_data = data[data["type_carriere"] == carriere].dropna(subset=["Longueur_Totale"])
    
    if not carriere_data.empty:
        longueur_carriere.append({
            "Type": f"Carrière: {carriere}",
            "Nombre_de_Posts": len(carriere_data),
            "Longueur_Moyenne": carriere_data["Longueur_Totale"].mean(),
            "Longueur_Minimale": carriere_data["Longueur_Totale"].min(),
            "Longueur_Maximale": carriere_data["Longueur_Totale"].max(),
            "Ecart_Type_Longueur": carriere_data["Longueur_Totale"].std(),
            "Longueur_Mediane": carriere_data["Longueur_Totale"].median(),
            "Pourcentage_Annexes": (carriere_data["Presence_Annexes"].sum() / len(carriere_data)) * 100,
            "Nombres d'Annexes": carriere_data["Nombre_Annexes"].sum()
        })

longueur_carriere = pd.DataFrame(longueur_carriere)
longueur_carriere

,Type,Nombre_de_Posts,Longueur_Moyenne,Longueur_Minimale,Longueur_Maximale,Ecart_Type_Longueur,Longueur_Mediane,Pourcentage_Annexes,Nombres d'Annexes
0,Carrière: loisir,117,1037.410256,13,2341,619.004333,1116.0,1.709402,2
1,Carrière: milieu,173,1534.433526,23,3122,638.702730,1534.0,6.936416,12
2,Carrière: semi_professionnelle,306,1430.905229,6,3778,934.451476,1419.5,22.549020,84


## Mentions dans les posts 

On va ici analyser l'usage que nos enquêtés font des mentions (d'auteurs, de maisons d'éditions, de leurs pairs...) à l'aide de @, au sein de leurs posts. 

In [93]:
# Analyse pour contenu et annexes
for zone, serie in [("Contenu", data["Contenu_Post"]), ("Annexes", data["Annexes_Post"])]:
    mentions = serie.apply(analyser_mentions)
    for key in mentions.iloc[0].keys():  # récupère toutes les clés du dict
        data[f"{key}_{zone}"] = mentions.apply(lambda x: x[key])

# Colonnes totales (booléens)
data["Contient_Mention"] = data["Contient_Mentions_Contenu"] | data["Contient_Mentions_Annexes"]
data["Contient_Mentions_Pairs_Total"] = data["Contient_Mentions_Pairs_Contenu"] | data["Contient_Mentions_Pairs_Annexes"]
data["Contient_Mentions_Auteurs_Total"] = data["Contient_Mentions_Auteurs_Contenu"] | data["Contient_Mentions_Auteurs_Annexes"]
data["Contient_Mentions_Autres_Total"] = data["Contient_Mentions_Autres_Contenu"] | data["Contient_Mentions_Autres_Annexes"]
data["Contient_Mentions_Maisons_Total"] = data["Contient_Mentions_Maisons_Contenu"] | data["Contient_Mentions_Maisons_Annexes"]

# Colonnes totales (nombres)
data["Nombre_Mentions_Total"] = data["Nombre_Mentions_Contenu"] + data["Nombre_Mentions_Annexes"]
data["Mentions_Auteurs_Total"] = data["Mentions_Auteurs_Contenu"] + data["Mentions_Auteurs_Annexes"]
data["Mentions_Maisons_Total"] = data["Mentions_Maisons_Contenu"] + data["Mentions_Maisons_Annexes"]
data["Mentions_Autres_Total"] = data["Mentions_Autres_Contenu"] + data["Mentions_Autres_Annexes"]
data["Mentions_Pairs_Total"] = data["Mentions_Pairs_Contenu"] + data["Mentions_Pairs_Annexes"]

data.head()


,Nom_d_emprunt,Contenu_Post,Code_Post,Code,Nombre,Annexes_Post,Date_Post,N_Video,N_Audio,N_Ecrit,...,Contient_Mention,Contient_Mentions_Pairs_Total,Contient_Mentions_Auteurs_Total,Contient_Mentions_Autres_Total,Contient_Mentions_Maisons_Total,Nombre_Mentions_Total,Mentions_Auteurs_Total,Mentions_Maisons_Total,Mentions_Autres_Total,Mentions_Pairs_Total
ID_Post,,,,,,,,,,,,,,,,,,,,,
1,Agathe,— 𝑳𝒆𝒄𝒕𝒖𝒓𝒆 : 𝑴𝒆𝒔 𝒑𝒍𝒖𝒔 𝒈𝒓𝒐𝒔 𝒄𝒐𝒖𝒑𝒔 𝒅𝒆 𝒄œ𝒖𝒓 —\n\nA...,F2,F,2,nan,2024-09-28,0,0,1,...,False,False,False,False,False,0,0,0,0,0
2,Agathe,"— 𝑮𝒆𝒕 𝒕𝒐 𝒌𝒏𝒐𝒘 𝒕𝒉𝒆 𝒓𝒆𝒂𝒅𝒆𝒓 —\n\nParce que oui, m...",F1,F,1,nan,2024-10-21,0,0,1,...,False,False,False,False,False,0,0,0,0,0
3,Alice,📍Et donc débuter par Lautréamont s'imposait.\n...,F1,F,1,nan,2024-04-10,0,0,1,...,False,False,False,False,False,0,0,0,0,0
4,Alice,“les mots qui cherchent\nles mots qui ne trouv...,F2,F,2,nan,2024-04-12,0,0,1,...,False,False,False,False,False,0,0,0,0,0
5,Alice,📍Le dernier livre d'Antoine Volodine – paru au...,F3,F,3,nan,2024-04-14,0,0,1,...,False,False,False,False,False,0,0,0,0,0


### Analyse des mentions

In [94]:
# Regrouper par personne et par période pour calculer les statistiques
mentions_enquete_period = (
    data.groupby(["Nom_d_emprunt", "Periode"])
    .agg(
        Nombre_Total_Mentions=("Nombre_Mentions_Total", "sum"),
        Mentions_Auteurs_Total=("Mentions_Auteurs_Total", "sum"),
        Mentions_Maisons_Total=("Mentions_Maisons_Total", "sum"),
        Mentions_Pairs_Total=("Mentions_Pairs_Total", "sum"),
        Mentions_Autres_Total=("Mentions_Autres_Total", "sum"),
        Nombre_Mentions_Contenu=("Nombre_Mentions_Contenu", "sum"),
        Nombre_Mentions_Annexes=("Nombre_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Contenu=("Contient_Mentions_Contenu", "sum"),
        Posts_Contenant_Mentions_Annexes=("Contient_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Total=("Contient_Mention", "sum"),
        Posts_Contenant_Mentions_Pairs=("Contient_Mentions_Pairs_Total", "sum"),
        Posts_Contenant_Mentions_Auteurs=("Contient_Mentions_Auteurs_Total", "sum"),
        Posts_Contenant_Mentions_Autres=("Contient_Mentions_Autres_Total", "sum"),
        Posts_Contenant_Mentions_Maisons=("Contient_Mentions_Maisons_Total", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  # Nombre total de posts
    )
    .reset_index()
)

In [95]:
# Calculer les pourcentages
cols_mentions = [
    "Posts_Contenant_Mentions_Contenu",
    "Posts_Contenant_Mentions_Annexes",
    "Posts_Contenant_Mentions_Pairs",
    "Posts_Contenant_Mentions_Auteurs",
    "Posts_Contenant_Mentions_Autres",
    "Posts_Contenant_Mentions_Maisons"
]

# Calcul automatique des pourcentages
for col in cols_mentions:
    mentions_enquete_period[f"Pourcentage_{col}"] = (
        mentions_enquete_period[col] /
        mentions_enquete_period["Nombre_de_Posts"] * 100
    )

# Cas spécial : total contenu + annexes
mentions_enquete_period["Pourcentage_Posts_Contenant_Mentions_Total"] = (
    (mentions_enquete_period["Posts_Contenant_Mentions_Contenu"] +
     mentions_enquete_period["Posts_Contenant_Mentions_Annexes"]) /
    mentions_enquete_period["Nombre_de_Posts"] * 100
)
mentions_enquete_period

,Nom_d_emprunt,Periode,Nombre_Total_Mentions,Mentions_Auteurs_Total,Mentions_Maisons_Total,Mentions_Pairs_Total,Mentions_Autres_Total,Nombre_Mentions_Contenu,Nombre_Mentions_Annexes,Posts_Contenant_Mentions_Contenu,...,Posts_Contenant_Mentions_Autres,Posts_Contenant_Mentions_Maisons,Nombre_de_Posts,Pourcentage_Posts_Contenant_Mentions_Contenu,Pourcentage_Posts_Contenant_Mentions_Annexes,Pourcentage_Posts_Contenant_Mentions_Pairs,Pourcentage_Posts_Contenant_Mentions_Auteurs,Pourcentage_Posts_Contenant_Mentions_Autres,Pourcentage_Posts_Contenant_Mentions_Maisons,Pourcentage_Posts_Contenant_Mentions_Total
0,Agathe,1,0,0,0,0,0,0,0,0,...,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alice,1,1,0,0,1,0,1,0,1,...,0,0,10,10.0,0.0,10.0,0.0,0.0,0.0,10.0
2,Alice,2,4,0,0,3,1,4,0,4,...,1,0,10,40.0,0.0,30.0,0.0,10.0,0.0,40.0
3,Alice,4,2,0,0,1,1,2,0,2,...,1,0,10,20.0,0.0,10.0,0.0,10.0,0.0,20.0
4,Aline,1,4,1,2,0,1,4,0,4,...,1,2,10,40.0,0.0,0.0,10.0,10.0,20.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Maude,4,11,2,9,0,0,11,0,9,...,0,9,10,90.0,0.0,0.0,20.0,0.0,90.0,90.0
65,Salomé,1,4,0,4,0,0,4,0,4,...,0,4,10,40.0,0.0,0.0,0.0,0.0,40.0,40.0
66,Salomé,2,11,0,1,8,2,11,0,9,...,2,1,10,90.0,0.0,70.0,0.0,20.0,10.0,90.0
67,Salomé,4,10,0,0,8,2,10,0,4,...,2,0,10,40.0,0.0,30.0,0.0,20.0,0.0,40.0


In [96]:
mentions_period = (
    data.groupby(["Periode"])
    .agg(
        Nombre_Total_Mentions=("Nombre_Mentions_Total", "sum"),
        Mentions_Auteurs_Total=("Mentions_Auteurs_Total", "sum"),
        Mentions_Maisons_Total=("Mentions_Maisons_Total", "sum"),
        Mentions_Pairs_Total=("Mentions_Pairs_Total", "sum"),
        Mentions_Autres_Total=("Mentions_Autres_Total", "sum"),
        Nombre_Mentions_Contenu=("Nombre_Mentions_Contenu", "sum"),
        Nombre_Mentions_Annexes=("Nombre_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Contenu=("Contient_Mentions_Contenu", "sum"),
        Posts_Contenant_Mentions_Annexes=("Contient_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Total=("Contient_Mention", "sum"),
        Posts_Contenant_Mentions_Pairs=("Contient_Mentions_Pairs_Total", "sum"),
        Posts_Contenant_Mentions_Auteurs=("Contient_Mentions_Auteurs_Total", "sum"),
        Posts_Contenant_Mentions_Autres=("Contient_Mentions_Autres_Total", "sum"),
        Posts_Contenant_Mentions_Maisons=("Contient_Mentions_Maisons_Total", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  # Nombre total de posts
    )
    .reset_index()
)

In [97]:
# Calculer les pourcentages
cols_mentions = [
    "Posts_Contenant_Mentions_Contenu",
    "Posts_Contenant_Mentions_Annexes",
    "Posts_Contenant_Mentions_Pairs",
    "Posts_Contenant_Mentions_Auteurs",
    "Posts_Contenant_Mentions_Autres",
    "Posts_Contenant_Mentions_Maisons"
]

# Calcul automatique des pourcentages
for col in cols_mentions:
    mentions_period[f"Pourcentage_{col}"] = (
        mentions_period[col] /
        mentions_period["Nombre_de_Posts"] * 100
    )

# les totaux
mentions_period["Pourcentage_Posts_Contenant_Mentions_Total"] = (
    (mentions_period["Posts_Contenant_Mentions_Contenu"] +
     mentions_period["Posts_Contenant_Mentions_Annexes"]) /
    mentions_period["Nombre_de_Posts"] * 100
)
mentions_period

,Periode,Nombre_Total_Mentions,Mentions_Auteurs_Total,Mentions_Maisons_Total,Mentions_Pairs_Total,Mentions_Autres_Total,Nombre_Mentions_Contenu,Nombre_Mentions_Annexes,Posts_Contenant_Mentions_Contenu,Posts_Contenant_Mentions_Annexes,...,Posts_Contenant_Mentions_Autres,Posts_Contenant_Mentions_Maisons,Nombre_de_Posts,Pourcentage_Posts_Contenant_Mentions_Contenu,Pourcentage_Posts_Contenant_Mentions_Annexes,Pourcentage_Posts_Contenant_Mentions_Pairs,Pourcentage_Posts_Contenant_Mentions_Auteurs,Pourcentage_Posts_Contenant_Mentions_Autres,Pourcentage_Posts_Contenant_Mentions_Maisons,Pourcentage_Posts_Contenant_Mentions_Total
0,1,113,38,53,5,17,109,4,67,3,...,14,45,182,36.813187,1.648352,2.197802,14.285714,7.692308,24.725275,38.461538
1,2,252,84,99,44,25,237,15,99,6,...,21,69,180,55.000000,3.333333,11.666667,23.888889,11.666667,38.333333,58.333333
2,3,101,31,58,9,3,86,15,20,8,...,2,27,40,50.000000,20.000000,7.500000,55.000000,5.000000,67.500000,70.000000
3,4,217,64,108,21,24,199,18,97,10,...,19,86,172,56.395349,5.813953,5.813953,26.744186,11.046512,50.000000,62.209302
4,E,27,12,5,1,9,21,6,10,2,...,5,5,22,45.454545,9.090909,4.545455,22.727273,22.727273,22.727273,54.545455


In [98]:
mentions_enquete = (
    data.groupby(["Nom_d_emprunt"])
    .agg(
        Nombre_Total_Mentions=("Nombre_Mentions_Total", "sum"),
        Mentions_Auteurs_Total=("Mentions_Auteurs_Total", "sum"),
        Mentions_Maisons_Total=("Mentions_Maisons_Total", "sum"),
        Mentions_Pairs_Total=("Mentions_Pairs_Total", "sum"),
        Mentions_Autres_Total=("Mentions_Autres_Total", "sum"),
        Nombre_Mentions_Contenu=("Nombre_Mentions_Contenu", "sum"),
        Nombre_Mentions_Annexes=("Nombre_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Contenu=("Contient_Mentions_Contenu", "sum"),
        Posts_Contenant_Mentions_Annexes=("Contient_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Total=("Contient_Mention", "sum"),
        Posts_Contenant_Mentions_Pairs=("Contient_Mentions_Pairs_Total", "sum"),
        Posts_Contenant_Mentions_Auteurs=("Contient_Mentions_Auteurs_Total", "sum"),
        Posts_Contenant_Mentions_Autres=("Contient_Mentions_Autres_Total", "sum"),
        Posts_Contenant_Mentions_Maisons=("Contient_Mentions_Maisons_Total", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  # Nombre total de posts
    )
    .reset_index()
)
# Calculer les pourcentages
cols_mentions = [
    "Posts_Contenant_Mentions_Contenu",
    "Posts_Contenant_Mentions_Annexes",
    "Posts_Contenant_Mentions_Pairs",
    "Posts_Contenant_Mentions_Auteurs",
    "Posts_Contenant_Mentions_Autres",
    "Posts_Contenant_Mentions_Maisons"
]

# Calcul automatique des pourcentages
for col in cols_mentions:
    mentions_enquete[f"Pourcentage_{col}"] = (
        mentions_enquete[col] /
        mentions_enquete["Nombre_de_Posts"] * 100
    )

# les totaux
mentions_enquete["Pourcentage_Posts_Contenant_Mentions_Total"] = (
    (mentions_enquete["Posts_Contenant_Mentions_Contenu"] +
     mentions_enquete["Posts_Contenant_Mentions_Annexes"]) /
    mentions_enquete["Nombre_de_Posts"] * 100
)
mentions_enquete

,Nom_d_emprunt,Nombre_Total_Mentions,Mentions_Auteurs_Total,Mentions_Maisons_Total,Mentions_Pairs_Total,Mentions_Autres_Total,Nombre_Mentions_Contenu,Nombre_Mentions_Annexes,Posts_Contenant_Mentions_Contenu,Posts_Contenant_Mentions_Annexes,...,Posts_Contenant_Mentions_Autres,Posts_Contenant_Mentions_Maisons,Nombre_de_Posts,Pourcentage_Posts_Contenant_Mentions_Contenu,Pourcentage_Posts_Contenant_Mentions_Annexes,Pourcentage_Posts_Contenant_Mentions_Pairs,Pourcentage_Posts_Contenant_Mentions_Auteurs,Pourcentage_Posts_Contenant_Mentions_Autres,Pourcentage_Posts_Contenant_Mentions_Maisons,Pourcentage_Posts_Contenant_Mentions_Total
0,Agathe,0,0,0,0,0,0,0,0,0,...,0,0,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Alice,7,0,0,5,2,7,0,7,0,...,2,0,30,23.333333,0.000000,16.666667,0.000000,6.666667,0.000000,23.333333
2,Aline,147,34,73,35,5,147,0,34,0,...,4,31,40,85.000000,0.000000,32.500000,52.500000,10.000000,77.500000,85.000000
3,Alya,0,0,0,0,0,0,0,0,0,...,0,0,30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Amélie,72,46,13,0,13,72,0,27,0,...,10,11,32,84.375000,0.000000,0.000000,59.375000,31.250000,34.375000,84.375000
5,Ariane,6,0,4,0,2,6,0,6,0,...,2,4,30,20.000000,0.000000,0.000000,0.000000,6.666667,13.333333,20.000000
6,Arthur,4,0,3,0,1,4,0,2,0,...,1,2,30,6.666667,0.000000,0.000000,0.000000,3.333333,6.666667,6.666667
7,Capucine,29,6,18,2,3,29,0,18,0,...,3,18,33,54.545455,0.000000,6.060606,18.181818,9.090909,54.545455,54.545455
8,Christine,64,30,28,0,6,22,42,7,23,...,4,23,41,17.073171,56.097561,0.000000,53.658537,9.756098,56.097561,73.170732
9,Claire,30,17,7,1,5,30,0,18,0,...,3,7,33,54.545455,0.000000,3.030303,36.363636,9.090909,21.212121,54.545455


In [99]:
mentions_enquete['type_carriere'] = mentions_enquete['Nom_d_emprunt'].apply(get_type_carriere)
# Regroupement par type_carriere avec somme pour les compteurs
mentions_carriere = mentions_enquete.groupby("type_carriere").agg({
    "Nombre_Total_Mentions": "sum",
    "Mentions_Auteurs_Total": "sum",
    "Mentions_Maisons_Total": "sum",
    "Mentions_Pairs_Total": "sum",
    "Mentions_Autres_Total": "sum",
    "Nombre_Mentions_Contenu": "sum",
    "Nombre_Mentions_Annexes": "sum",
    "Posts_Contenant_Mentions_Contenu": "sum",
    "Posts_Contenant_Mentions_Annexes": "sum",
    "Posts_Contenant_Mentions_Total": "sum",
    "Posts_Contenant_Mentions_Pairs": "sum",
    "Posts_Contenant_Mentions_Auteurs": "sum",
    "Posts_Contenant_Mentions_Autres": "sum",
    "Posts_Contenant_Mentions_Maisons": "sum",
    "Nombre_de_Posts": "sum",
})

# Calculer les pourcentages moyens pondérés par nombre de posts (optionnel)
# Par exemple : moyenne des pourcentages, pondérée par nombre de posts
for col in [
    "Pourcentage_Posts_Contenant_Mentions_Contenu",
    "Pourcentage_Posts_Contenant_Mentions_Annexes",
    "Pourcentage_Posts_Contenant_Mentions_Total"
]:
    mentions_carriere[col + "_moyenne_ponderee"] = (
        (mentions_enquete[col] * mentions_enquete["Nombre_de_Posts"]).groupby(mentions_enquete["type_carriere"]).sum()
        / mentions_enquete.groupby("type_carriere")["Nombre_de_Posts"].sum()
    )

mentions_carriere


,Nombre_Total_Mentions,Mentions_Auteurs_Total,Mentions_Maisons_Total,Mentions_Pairs_Total,Mentions_Autres_Total,Nombre_Mentions_Contenu,Nombre_Mentions_Annexes,Posts_Contenant_Mentions_Contenu,Posts_Contenant_Mentions_Annexes,Posts_Contenant_Mentions_Total,Posts_Contenant_Mentions_Pairs,Posts_Contenant_Mentions_Auteurs,Posts_Contenant_Mentions_Autres,Posts_Contenant_Mentions_Maisons,Nombre_de_Posts,Pourcentage_Posts_Contenant_Mentions_Contenu_moyenne_ponderee,Pourcentage_Posts_Contenant_Mentions_Annexes_moyenne_ponderee,Pourcentage_Posts_Contenant_Mentions_Total_moyenne_ponderee
type_carriere,,,,,,,,,,,,,,,,,,
loisir,36,3,5,21,7,36,0,28,0,28,15,3,7,5,117,23.931624,0.000000,23.931624
milieu,280,97,134,39,10,280,0,99,0,99,16,56,9,85,173,57.225434,0.000000,57.225434
semi_professionnelle,394,129,184,20,61,336,58,166,29,189,8,83,45,142,306,54.248366,9.477124,63.725490


In [100]:
# Calcul des statistiques globales
stats_globales = {
    "Nombre_Total_Posts": len(data),
    "Nombre_Total_Mentions": data["Nombre_Mentions_Total"].sum(),
    "Mentions_Auteurs_Total": data["Mentions_Auteurs_Total"].sum(),
    "Mentions_Maisons_Total": data["Mentions_Maisons_Total"].sum(),
    "Mentions_Pairs_Total": data["Mentions_Pairs_Total"].sum(),
    "Mentions_Autres_Total": data["Mentions_Autres_Total"].sum(),
    "Pourcentage_Posts_Contenant_Mentions_": data["Contient_Mentions_Contenu"].sum() / len(data) * 100,
    "Pourcentage_Posts_Contenant_Mentions_Annexes": data["Contient_Mentions_Annexes"].sum() / len(data) * 100,
    "Pourcentage_Posts_Contenant_Mentions_Total": (data["Contient_Mention"].sum() / len(data)) * 100,
    "Posts_Contenant_Mentions_Contenu": data["Contient_Mentions_Contenu"].sum(),
    "Posts_Contenant_Mentions_Annexes": data["Contient_Mentions_Annexes"].sum(),
    "Posts_Contenant_Mentions_Total": data["Contient_Mention"].sum(),
    "Posts_Contenant_Mentions_Pairs": data["Contient_Mentions_Pairs_Total"].sum(),
    "Posts_Contenant_Mentions_Auteurs": data["Contient_Mentions_Auteurs_Total"].sum(),
    "Posts_Contenant_Mentions_Autres": data["Contient_Mentions_Autres_Total"].sum(),
    "Posts_Contenant_Mentions_Maisons": data["Contient_Mentions_Maisons_Total"].sum(),
}
    

# Afficher les statistiques globales
for key, value in stats_globales.items():
    print(f"{key}: {value}")

Nombre_Total_Posts: 596
Nombre_Total_Mentions: 710
Mentions_Auteurs_Total: 229
Mentions_Maisons_Total: 323
Mentions_Pairs_Total: 80
Mentions_Autres_Total: 78
Pourcentage_Posts_Contenant_Mentions_: 49.16107382550335
Pourcentage_Posts_Contenant_Mentions_Annexes: 4.865771812080537
Pourcentage_Posts_Contenant_Mentions_Total: 53.02013422818792
Posts_Contenant_Mentions_Contenu: 293
Posts_Contenant_Mentions_Annexes: 29
Posts_Contenant_Mentions_Total: 316
Posts_Contenant_Mentions_Pairs: 39
Posts_Contenant_Mentions_Auteurs: 142
Posts_Contenant_Mentions_Autres: 61
Posts_Contenant_Mentions_Maisons: 232


## Hashtag dans les posts

In [101]:
# Analyse pour contenu et annexes
for zone, serie in [("Contenu", data["Contenu_Post"]), ("Annexes", data["Annexes_Post"])]:
    mentions = serie.apply(analyser_hashtags)
    for key in mentions.iloc[0].keys():  # récupère toutes les clés du dict
        data[f"{key}_{zone}"] = mentions.apply(lambda x: x[key])

# Calcul des totaux (hashtags contenu + annexes)
data["Nombre_Hashtags_Total"] = data["Nombre_Hashtags_Contenu"] + data["Nombre_Hashtags_Annexes"]
data["Contient_Hashtags"] = data["Contient_Hashtags_Contenu"] | data["Contient_Hashtags_Annexes"]
data.head()

,Nom_d_emprunt,Contenu_Post,Code_Post,Code,Nombre,Annexes_Post,Date_Post,N_Video,N_Audio,N_Ecrit,...,Mentions_Autres_Total,Mentions_Pairs_Total,Contient_Hashtags_Contenu,Nombre_Hashtags_Contenu,Liste_Hashtags_Contenu,Contient_Hashtags_Annexes,Nombre_Hashtags_Annexes,Liste_Hashtags_Annexes,Nombre_Hashtags_Total,Contient_Hashtags
ID_Post,,,,,,,,,,,,,,,,,,,,,
1,Agathe,— 𝑳𝒆𝒄𝒕𝒖𝒓𝒆 : 𝑴𝒆𝒔 𝒑𝒍𝒖𝒔 𝒈𝒓𝒐𝒔 𝒄𝒐𝒖𝒑𝒔 𝒅𝒆 𝒄œ𝒖𝒓 —\n\nA...,F2,F,2,nan,2024-09-28,0,0,1,...,0,0,True,30,"[autrice, autricefrançaise, écriture, ecrire, ...",False,0,[],30,True
2,Agathe,"— 𝑮𝒆𝒕 𝒕𝒐 𝒌𝒏𝒐𝒘 𝒕𝒉𝒆 𝒓𝒆𝒂𝒅𝒆𝒓 —\n\nParce que oui, m...",F1,F,1,nan,2024-10-21,0,0,1,...,0,0,True,30,"[autrice, autricefrançaise, écriture, ecrire, ...",False,0,[],30,True
3,Alice,📍Et donc débuter par Lautréamont s'imposait.\n...,F1,F,1,nan,2024-04-10,0,0,1,...,0,0,True,7,"[lautréamont, leschantsdemaldoror, surréalisme...",False,0,[],7,True
4,Alice,“les mots qui cherchent\nles mots qui ne trouv...,F2,F,2,nan,2024-04-12,0,0,1,...,0,0,True,5,"[nannibalestrini, chaosmogonie, editionslatemp...",False,0,[],5,True
5,Alice,📍Le dernier livre d'Antoine Volodine – paru au...,F3,F,3,nan,2024-04-14,0,0,1,...,0,0,True,6,"[volodine, seuil, postexotisme, post, vivredan...",False,0,[],6,True


### Analyse des hastags

In [102]:
# Regrouper par personne et par période pour calculer les statistiques
hashtags_enquete_periode = (
    data.groupby(["Nom_d_emprunt", "Periode"])
    .agg(
        Nombre_Total_Hashtags=("Nombre_Hashtags_Total", "sum"),
        Nombre_Hashtags_Contenu=("Nombre_Hashtags_Contenu", "sum"),
        Nombre_Hashtags_Annexes=("Nombre_Hashtags_Annexes", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  # Nombre total de posts
    )
    .reset_index()
)

# Calculer le pourcentage de posts contenant des hashtags
hashtags_enquete_periode["Pourcentage_Posts_Contenant_Hashtags_Contenu"] = (
    (data["Nombre_Hashtags_Contenu"] > 0).groupby([data["Nom_d_emprunt"], data["Periode"]]).sum() /
    hashtags_enquete_periode.set_index(["Nom_d_emprunt", "Periode"])["Nombre_de_Posts"] * 100
).values
hashtags_enquete_periode["Pourcentage_Posts_Contenant_Hashtags_Annexes"] = (
    (data["Nombre_Hashtags_Annexes"] > 0).groupby([data["Nom_d_emprunt"], data["Periode"]]).sum() /
    hashtags_enquete_periode.set_index(["Nom_d_emprunt", "Periode"])["Nombre_de_Posts"] * 100
).values
hashtags_enquete_periode["Pourcentage_Posts_Contenant_Hashtags_Total"] = (
    ((data["Nombre_Hashtags_Contenu"] > 0) | (data["Nombre_Hashtags_Annexes"] > 0)).groupby([data["Nom_d_emprunt"], data["Periode"]]).sum() /
    hashtags_enquete_periode.set_index(["Nom_d_emprunt", "Periode"])["Nombre_de_Posts"] * 100
).values

hashtags_enquete_periode

,Nom_d_emprunt,Periode,Nombre_Total_Hashtags,Nombre_Hashtags_Contenu,Nombre_Hashtags_Annexes,Nombre_de_Posts,Pourcentage_Posts_Contenant_Hashtags_Contenu,Pourcentage_Posts_Contenant_Hashtags_Annexes,Pourcentage_Posts_Contenant_Hashtags_Total
0,Agathe,1,60,60,0,2,100.0,0.0,100.0
1,Alice,1,54,54,0,10,100.0,0.0,100.0
2,Alice,2,66,66,0,10,100.0,0.0,100.0
3,Alice,4,68,68,0,10,100.0,0.0,100.0
4,Aline,1,0,0,0,10,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
64,Maude,4,0,0,0,10,0.0,0.0,0.0
65,Salomé,1,42,42,0,10,100.0,0.0,100.0
66,Salomé,2,118,118,0,10,100.0,0.0,100.0
67,Salomé,4,95,84,11,10,90.0,10.0,100.0


In [103]:
hashtags_enquete = (
    data.groupby("Nom_d_emprunt")
    .agg(
        Nombre_Total_Hashtags=("Nombre_Hashtags_Total", "sum"),
        Nombre_Hashtags_Contenu=("Nombre_Hashtags_Contenu", "sum"),
        Nombre_Hashtags_Annexes=("Nombre_Hashtags_Annexes", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  # Total de posts par personne
    )
    .reset_index()
)

# Calcul du pourcentage de posts avec hashtags pour chaque enquêté
hashtags_enquete["Pourcentage_Posts_Contenant_Hashtags_Contenu"] = (
    (data["Nombre_Hashtags_Contenu"] > 0).groupby(data["Nom_d_emprunt"]).sum() /
    hashtags_enquete.set_index("Nom_d_emprunt")["Nombre_de_Posts"] * 100
).values
hashtags_enquete["Pourcentage_Posts_Contenant_Hashtags_Annexes"] = (
    (data["Nombre_Hashtags_Annexes"] > 0).groupby(data["Nom_d_emprunt"]).sum() /
    hashtags_enquete.set_index("Nom_d_emprunt")["Nombre_de_Posts"] * 100
).values
hashtags_enquete["Pourcentage_Posts_Contenant_Hashtags_Total"] = (
    ((data["Nombre_Hashtags_Contenu"] > 0) | (data["Nombre_Hashtags_Annexes"] > 0))
    .groupby(data["Nom_d_emprunt"]).sum() /
    hashtags_enquete.set_index("Nom_d_emprunt")["Nombre_de_Posts"] * 100
).values

hashtags_enquete


,Nom_d_emprunt,Nombre_Total_Hashtags,Nombre_Hashtags_Contenu,Nombre_Hashtags_Annexes,Nombre_de_Posts,Pourcentage_Posts_Contenant_Hashtags_Contenu,Pourcentage_Posts_Contenant_Hashtags_Annexes,Pourcentage_Posts_Contenant_Hashtags_Total
0,Agathe,60,60,0,2,100.000000,0.000000,100.000000
1,Alice,188,188,0,30,100.000000,0.000000,100.000000
2,Aline,423,423,0,40,75.000000,0.000000,75.000000
3,Alya,160,160,0,30,66.666667,0.000000,66.666667
4,Amélie,393,388,5,32,68.750000,3.125000,68.750000
5,Ariane,274,270,4,30,93.333333,3.333333,96.666667
6,Arthur,844,844,0,30,100.000000,0.000000,100.000000
7,Capucine,608,608,0,33,96.969697,0.000000,96.969697
8,Christine,769,182,587,41,34.146341,56.097561,90.243902
9,Claire,136,136,0,33,75.757576,0.000000,75.757576


In [104]:
hashtags_enquete['type_carriere'] = hashtags_enquete['Nom_d_emprunt'].apply(get_type_carriere)

In [105]:
# Analyse des hastags par type de carrière
hashtags_carriere = (
    hashtags_enquete
    .groupby("type_carriere")
    .agg(
        Nombre_Total_Hashtags=("Nombre_Total_Hashtags", "sum"),
        Nombre_Hashtags_Contenu=("Nombre_Hashtags_Contenu", "sum"),
        Nombre_Hashtags_Annexes=("Nombre_Hashtags_Annexes", "sum"),
        Nombre_de_Posts=("Nombre_de_Posts", "sum"),
        Pourcentage_Moyen_Posts_Contenant_Hashtags_Contenu=("Pourcentage_Posts_Contenant_Hashtags_Contenu", "mean"),
        Pourcentage_Moyen_Posts_Contenant_Hashtags_Annexes=("Pourcentage_Posts_Contenant_Hashtags_Annexes", "mean"),
        Pourcentage_Moyen_Posts_Contenant_Hashtags_Total=("Pourcentage_Posts_Contenant_Hashtags_Total", "mean"),
    )
    .reset_index()
)

hashtags_carriere = hashtags_carriere.set_index("type_carriere")
hashtags_carriere


,Nombre_Total_Hashtags,Nombre_Hashtags_Contenu,Nombre_Hashtags_Annexes,Nombre_de_Posts,Pourcentage_Moyen_Posts_Contenant_Hashtags_Contenu,Pourcentage_Moyen_Posts_Contenant_Hashtags_Annexes,Pourcentage_Moyen_Posts_Contenant_Hashtags_Total
type_carriere,,,,,,,
loisir,745,734,11,117,91.213768,0.625000,91.838768
milieu,2052,1913,139,173,73.883721,5.519380,79.403101
semi_professionnelle,3847,3008,839,306,71.340732,17.595816,86.080365


In [106]:
# Analyse des hashtags par période
hashtags_periode = (
    data.groupby([ "Periode"])
    .agg(
        Nombre_Total_Hashtags=("Nombre_Hashtags_Total", "sum"),
        Nombre_Hashtags_Contenu=("Nombre_Hashtags_Contenu", "sum"),
        Nombre_Hashtags_Annexes=("Nombre_Hashtags_Annexes", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  
    )
    .reset_index()
)

# Calculer le pourcentage de posts contenant des hashtags
hashtags_periode["Pourcentage_Posts_Contenant_Hashtags_Contenu"] = (
    (data["Nombre_Hashtags_Contenu"] > 0).groupby([data["Periode"]]).sum() /
    hashtags_periode.set_index(["Periode"])["Nombre_de_Posts"] * 100
).values
hashtags_periode["Pourcentage_Posts_Contenant_Hashtags_Annexes"] = (
    (data["Nombre_Hashtags_Annexes"] > 0).groupby([data["Periode"]]).sum() /
    hashtags_periode.set_index(["Periode"])["Nombre_de_Posts"] * 100
).values
hashtags_periode["Pourcentage_Posts_Contenant_Hashtags_Total"] = (
    ((data["Nombre_Hashtags_Contenu"] > 0) | (data["Nombre_Hashtags_Annexes"] > 0)).groupby([data["Periode"]]).sum() /
    hashtags_periode.set_index(["Periode"])["Nombre_de_Posts"] * 100
).values

hashtags_periode

,Periode,Nombre_Total_Hashtags,Nombre_Hashtags_Contenu,Nombre_Hashtags_Annexes,Nombre_de_Posts,Pourcentage_Posts_Contenant_Hashtags_Contenu,Pourcentage_Posts_Contenant_Hashtags_Annexes,Pourcentage_Posts_Contenant_Hashtags_Total
0,1,1551,1445,106,182,76.373626,7.692308,81.868132
1,2,2341,2021,320,180,78.333333,10.000000,88.333333
2,3,514,293,221,40,55.000000,25.000000,80.000000
3,4,2039,1729,310,172,75.581395,12.790698,86.627907
4,E,199,167,32,22,72.727273,9.090909,77.272727


In [107]:
# Calcul des statistiques globales à propos des hashtags
hashtags_globales = {
    "Nombre_Total_Posts": len(data),
    "Nombre_Total_Hashtags": data["Nombre_Hashtags_Total"].sum(),
    "Nombre_Hashtags_Contenu": data["Nombre_Hashtags_Contenu"].sum(),
    "Nombre_Hashtags_Annexes": data["Nombre_Hashtags_Annexes"].sum(),
    "Nombre_Hashtags_Total": data["Nombre_Hashtags_Total"].sum(),
    "Pourcentage_Posts_Contenant_Hashtags_Contenu": data["Contient_Hashtags_Contenu"].sum() / len(data) * 100,
    "Pourcentage_Posts_Contenant_Hashtags_Annexes": data["Contient_Hashtags_Annexes"].sum() / len(data) * 100,
    "Pourcentage_Posts_Contenant_Hashtags_Total": (data["Contient_Hashtags"].sum() / len(data)) * 100,
    "Posts_Contenant_Hashtags_Contenu": data["Contient_Hashtags_Contenu"].sum(),
    "Posts_Contenant_Hashtags_Annexes": data["Contient_Hashtags_Annexes"].sum(),   
}
    
for key, value in hashtags_globales.items():
    print(f"{key}: {value}")

Nombre_Total_Posts: 596
Nombre_Total_Hashtags: 6644
Nombre_Hashtags_Contenu: 5655
Nombre_Hashtags_Annexes: 989
Nombre_Hashtags_Total: 6644
Pourcentage_Posts_Contenant_Hashtags_Contenu: 75.16778523489933
Pourcentage_Posts_Contenant_Hashtags_Annexes: 11.073825503355705
Pourcentage_Posts_Contenant_Hashtags_Total: 84.8993288590604
Posts_Contenant_Hashtags_Contenu: 448
Posts_Contenant_Hashtags_Annexes: 66


## Analyses des images 

In [108]:
# Liste des colonnes binaires
colonnes_binaires = ['Plusieurs_Images', 'Personne', 'PostProduction', 'Mots']
images_globales = {}

for col in colonnes_binaires:
    images_globales[col] = {
        'Présence': data[col].sum(),
        'Absence': data[col].count() - data[col].sum(),
        'Pourcentage_Présence': data[col].mean() * 100,
        'Total': data[col].count()
    }

images_globales = pd.DataFrame(images_globales)
images_globales

# lister les posts sur lesquels il n'y a pas ces informations pour vérifier les erreurs potentielles
#posts_sans_info = data[data[colonnes_binaires].isnull().any(axis=1)][['ID_Post', 'Nom_d_emprunt', 'Periode', 'Date_Post'] + colonnes_binaires]
#posts_sans_info

,Plusieurs_Images,Personne,PostProduction,Mots
Présence,191.000000,37.00000,215.000000,198.000000
Absence,371.000000,525.00000,347.000000,364.000000
Pourcentage_Présence,33.985765,6.58363,38.256228,35.231317
Total,562.000000,562.00000,562.000000,562.000000


In [109]:
# Analyse par personne des images
images_enquete = data.groupby('Nom_d_emprunt')[colonnes_binaires].sum()
images_enquete['Total_Posts'] = data.groupby('Nom_d_emprunt').size()
images_enquete = images_enquete.div(images_enquete['Total_Posts'], axis=0) * 100

images_enquete

,Plusieurs_Images,Personne,PostProduction,Mots,Total_Posts
Nom_d_emprunt,,,,,
Agathe,0.000000,0.000000,0.000000,0.000000,100.0
Alice,0.000000,0.000000,0.000000,0.000000,100.0
Aline,0.000000,2.500000,67.500000,45.000000,100.0
Alya,100.000000,0.000000,100.000000,100.000000,100.0
Amélie,43.750000,18.750000,62.500000,53.125000,100.0
Ariane,3.333333,0.000000,10.000000,10.000000,100.0
Arthur,70.000000,0.000000,0.000000,0.000000,100.0
Capucine,0.000000,0.000000,9.090909,9.090909,100.0
Christine,12.195122,0.000000,7.317073,4.878049,100.0


In [110]:
# enlever les posts dont les images ne sont plus disponibles lors de la collecte des données
colonnes_image = ["Personne", "PostProduction", "Mots", "Plusieurs_Images"]
data_valable = data.dropna(subset=colonnes_image, how='all')

# Calculer les totaux des colonnes binaires, par période, sur ces données filtrées
images_periode = data_valable.groupby('Periode')[colonnes_binaires].sum()
images_periode['Total_Posts'] = data.groupby('Periode').size()

images_periode

,Plusieurs_Images,Personne,PostProduction,Mots,Total_Posts
Periode,,,,,
1,44.0,4.0,52.0,46.0,182
2,58.0,16.0,73.0,69.0,180
3,14.0,5.0,17.0,13.0,40
4,68.0,9.0,61.0,58.0,172
E,7.0,3.0,12.0,12.0,22


In [111]:
# Analyse par carrière des images
images_carriere = data.groupby('type_carriere')[colonnes_binaires].sum()
images_carriere['Total_Posts'] = data.groupby('type_carriere').size()
images_carriere = images_carriere.div(images_carriere['Total_Posts'], axis=0) * 100

images_carriere

,Plusieurs_Images,Personne,PostProduction,Mots,Total_Posts
type_carriere,,,,,
loisir,36.752137,0.854701,30.769231,30.769231,100.0
milieu,22.543353,0.578035,19.653179,14.450867,100.0
semi_professionnelle,35.620915,11.437908,47.385621,44.771242,100.0


In [112]:
# Analyse croisée par personne et période
images_enquete_periode = data.groupby(['Nom_d_emprunt', 'Periode'])[colonnes_binaires].sum()
images_enquete_periode['Total_Posts'] = data.groupby(['Nom_d_emprunt', 'Periode']).size()
images_enquete_periode = images_enquete_periode.div(images_enquete_periode['Total_Posts'], axis=0) * 100
images_enquete_periode


Plusieurs_Images  Personne  PostProduction  Mots  \
Nom_d_emprunt Periode                                                     
Agathe        1                     0.0       0.0             0.0   0.0   
Alice         1                     0.0       0.0             0.0   0.0   
              2                     0.0       0.0             0.0   0.0   
              4                     0.0       0.0             0.0   0.0   
Aline         1                     0.0      10.0            90.0  90.0   
...                                 ...       ...             ...   ...   
Maude         4                     0.0       0.0             0.0   0.0   
Salomé        1                    10.0       0.0             0.0   0.0   
              2                    70.0      10.0            60.0  60.0   
              4                    50.0       0.0             0.0   0.0   
              E                     0.0       0.0             0.0   0.0   

                       Total_Posts  
Nom_d_emprunt Periode               
Agathe        1              100.0  
Alice         1              100.0  
              2              100.0  
              4              100.0  
Aline         1              100.0  
...                            ...  
Maude         4              100.0  
Salomé        1              100.0  
              2              100.0  
              4              100.0  
              E              100.0  

[69 rows x 5 columns]

## Analyse de la nature des posts 

In [113]:
# Analyse globale
nature_globales = data[['N_Ecrit', 'N_Video', 'N_Audio']].sum().to_frame(name='Total')
nature_globales['Pourcentage'] = (nature_globales['Total'] /596) * 100
nature_globales['Total_Posts'] = len(data)

nature_globales

,Total,Pourcentage,Total_Posts
N_Ecrit,579,97.147651,596
N_Video,60,10.067114,596
N_Audio,7,1.174497,596


In [114]:
# Analyse par enquete
nature_enquete = data.groupby('Nom_d_emprunt')[['N_Ecrit', 'N_Video', 'N_Audio']].sum()
nature_enquete['Total_Posts'] = data.groupby('Nom_d_emprunt').size()
nature_enquete

,N_Ecrit,N_Video,N_Audio,Total_Posts
Nom_d_emprunt,,,,
Agathe,2,0,0,2
Alice,30,6,0,30
Aline,40,0,0,40
Alya,30,0,0,30
Amélie,31,9,0,32
Ariane,30,0,0,30
Arthur,30,0,0,30
Capucine,33,1,0,33
Christine,41,3,0,41


In [115]:
# Analyse par période
nature_periode = data.groupby('Periode')[['N_Ecrit', 'N_Video', 'N_Audio']].sum()
nature_periode['Total_Posts'] = data.groupby('Periode').size()
nature_periode = nature_periode.div(nature_periode['Total_Posts'], axis=0) * 100
nature_periode

,N_Ecrit,N_Video,N_Audio,Total_Posts
Periode,,,,
1,98.351648,8.791209,1.648352,100.0
2,96.666667,11.666667,1.666667,100.0
3,100.000000,15.000000,0.000000,100.0
4,95.348837,9.302326,0.581395,100.0
E,100.000000,4.545455,0.000000,100.0


In [116]:
# Analyse croisée par personne et période
nature_enquete_periode = data.groupby(['Nom_d_emprunt', 'Periode'])[['N_Ecrit', 'N_Video', 'N_Audio']].sum()
nature_enquete_periode['Total_Posts'] = data.groupby(['Nom_d_emprunt', 'Periode']).size()
nature_enquete_periode = nature_enquete_periode.div(nature_enquete_periode['Total_Posts'], axis=0) * 100
nature_enquete_periode

N_Ecrit  N_Video  N_Audio  Total_Posts
Nom_d_emprunt Periode                                        
Agathe        1          100.0      0.0      0.0        100.0
Alice         1          100.0     20.0      0.0        100.0
              2          100.0     20.0      0.0        100.0
              4          100.0     20.0      0.0        100.0
Aline         1          100.0      0.0      0.0        100.0
...                        ...      ...      ...          ...
Maude         4          100.0      0.0      0.0        100.0
Salomé        1          100.0      0.0      0.0        100.0
              2          100.0      0.0      0.0        100.0
              4          100.0      0.0      0.0        100.0
              E          100.0      0.0      0.0        100.0

[69 rows x 4 columns]

In [117]:
# Analyse par carriere
nature_carriere = data.groupby('type_carriere')[['N_Ecrit', 'N_Video', 'N_Audio']].sum()
nature_carriere['Total_Posts'] = data.groupby('type_carriere').size()
nature_carriere = nature_carriere.div(nature_carriere['Total_Posts'], axis=0) * 100

nature_carriere

,N_Ecrit,N_Video,N_Audio,Total_Posts
type_carriere,,,,
loisir,100.000000,5.128205,0.000000,100.0
milieu,91.907514,0.000000,0.000000,100.0
semi_professionnelle,99.019608,17.647059,2.287582,100.0


# Tableaux pour le clustering

In [118]:

supra_df = frequence.merge(enquete_longueur, on=["Nom_d_emprunt", "Periode"], how="left")
supra_df = supra_df.merge(mentions_enquete_period.drop(columns = "Nombre_de_Posts"), on=["Nom_d_emprunt", "Periode"], how="left")
supra_df = supra_df.merge(hashtags_enquete_periode, on=["Nom_d_emprunt", "Periode"], how="left")
supra_df = supra_df.merge(images_enquete_periode, on=["Nom_d_emprunt", "Periode"], how="left")
supra_df = supra_df.merge(nature_enquete_periode, on=["Nom_d_emprunt", "Periode"], how="left")

supra_df

,Nom_d_emprunt,Periode,Carrière,Nombre_de_Posts_x,Intervalle_Moyen,Intervalle_Minimum,Intervalle_Maximum,Intervalle_Median,Ecart_Type_Intervalle,Intervalle_Mediane,...,Pourcentage_Posts_Contenant_Hashtags_Total,Plusieurs_Images,Personne,PostProduction,Mots,Total_Posts_x,N_Ecrit,N_Video,N_Audio,Total_Posts_y
0,Agathe,1,['loisir'],2,23.000000,23.0,23.0,23.0,NaN,NaN,...,100.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,100.0
1,Alice,1,['loisir'],10,1.666667,0.0,3.0,2.0,0.866025,NaN,...,100.0,0.0,0.0,0.0,0.0,100.0,100.0,20.0,0.0,100.0
2,Alice,2,['loisir'],10,7.666667,2.0,23.0,5.0,6.538348,NaN,...,100.0,0.0,0.0,0.0,0.0,100.0,100.0,20.0,0.0,100.0
3,Alice,4,['loisir'],10,12.444444,2.0,23.0,12.0,7.763876,NaN,...,100.0,0.0,0.0,0.0,0.0,100.0,100.0,20.0,0.0,100.0
4,Aline,1,['milieu'],10,8.333333,0.0,68.0,1.0,22.377444,NaN,...,0.0,0.0,10.0,90.0,90.0,100.0,100.0,0.0,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Maude,4,['milieu'],10,19.111111,8.0,34.0,22.0,9.225568,NaN,...,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,100.0
63,Salomé,1,['loisir'],11,4.700000,0.0,9.0,4.0,2.790858,NaN,...,100.0,10.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,100.0
64,Salomé,2,['loisir'],10,5.777778,1.0,17.0,4.0,5.517648,NaN,...,100.0,70.0,10.0,60.0,60.0,100.0,100.0,0.0,0.0,100.0
65,Salomé,4,['loisir'],10,4.000000,1.0,10.0,3.0,3.278719,NaN,...,100.0,50.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,100.0


In [119]:
# Afficher la liste des noms de colonnes de merged_df
supra_df.columns.tolist()

['Nom_d_emprunt',
 'Periode',
 'Carrière',
 'Nombre_de_Posts_x',
 'Intervalle_Moyen',
 'Intervalle_Minimum',
 'Intervalle_Maximum',
 'Intervalle_Median',
 'Ecart_Type_Intervalle',
 'Intervalle_Mediane',
 'Nombre_de_Posts_y',
 'Longueur_Moyenne',
 'Longueur_Minimale',
 'Longueur_Maximale',
 'Ecart_Type_Longueur',
 'Longueur_Mediane',
 'Pourcentage_Annexes',
 "Nombres d'Annexes",
 'Nombre_Total_Mentions',
 'Mentions_Auteurs_Total',
 'Mentions_Maisons_Total',
 'Mentions_Pairs_Total',
 'Mentions_Autres_Total',
 'Nombre_Mentions_Contenu',
 'Nombre_Mentions_Annexes',
 'Posts_Contenant_Mentions_Contenu',
 'Posts_Contenant_Mentions_Annexes',
 'Posts_Contenant_Mentions_Total',
 'Posts_Contenant_Mentions_Pairs',
 'Posts_Contenant_Mentions_Auteurs',
 'Posts_Contenant_Mentions_Autres',
 'Posts_Contenant_Mentions_Maisons',
 'Pourcentage_Posts_Contenant_Mentions_Contenu',
 'Pourcentage_Posts_Contenant_Mentions_Annexes',
 'Pourcentage_Posts_Contenant_Mentions_Pairs',
 'Pourcentage_Posts_Contenant_Men

On visualise les colonnes de notre df et on enleve celles qui ne sont pas nécessaires pour le clustering

In [120]:
supra_df.drop(columns=[ 'Carrière',
 'Intervalle_Mediane',
 'Longueur_Minimale',
 'Longueur_Maximale',
 'Ecart_Type_Longueur',
 'Longueur_Mediane',
 "Nombres d'Annexes",
  'Intervalle_Minimum',
 'Intervalle_Maximum','Total_Posts_y','Total_Posts_x', 'Pourcentage_Posts_Contenant_Hashtags_Contenu',
 'Pourcentage_Posts_Contenant_Hashtags_Annexes', 'Nombre_Hashtags_Contenu',
 'Nombre_Hashtags_Annexes', 'Pourcentage_Posts_Contenant_Mentions_Contenu',
 'Pourcentage_Posts_Contenant_Mentions_Annexes', 'Nombre_Mentions_Contenu',
 'Nombre_Mentions_Annexes',
 'Posts_Contenant_Mentions_Contenu',
 'Posts_Contenant_Mentions_Annexes',
 'Nombre_de_Posts_y','Nombre_de_Posts_x',  'Mentions_Auteurs_Total',
 'Mentions_Maisons_Total',
 'Mentions_Autres_Total',
 'Nombre_Mentions_Contenu',
 'Nombre_Mentions_Annexes','Posts_Contenant_Mentions_Total',
 'Posts_Contenant_Mentions_Pairs',
 'Posts_Contenant_Mentions_Auteurs',
 'Posts_Contenant_Mentions_Autres',
 'Posts_Contenant_Mentions_Maisons','Pourcentage_Posts_Contenant_Mentions_Autres','Mentions_Pairs_Total'], inplace=True)

In [121]:
# Sauvegarder le DataFrame final pour le clustering
#merged_df.to_excel("Clustering_Data.xlsx", index=False)
supra_df.columns.tolist()

['Nom_d_emprunt',
 'Periode',
 'Intervalle_Moyen',
 'Intervalle_Median',
 'Ecart_Type_Intervalle',
 'Longueur_Moyenne',
 'Pourcentage_Annexes',
 'Nombre_Total_Mentions',
 'Pourcentage_Posts_Contenant_Mentions_Pairs',
 'Pourcentage_Posts_Contenant_Mentions_Auteurs',
 'Pourcentage_Posts_Contenant_Mentions_Maisons',
 'Pourcentage_Posts_Contenant_Mentions_Total',
 'Nombre_Total_Hashtags',
 'Nombre_de_Posts',
 'Pourcentage_Posts_Contenant_Hashtags_Total',
 'Plusieurs_Images',
 'Personne',
 'PostProduction',
 'Mots',
 'N_Ecrit',
 'N_Video',
 'N_Audio']

In [122]:
supra_df
# On obtient le DataFrame final avec les colonnes nécessaires pour le clustering des couples enquêtés et périodes.

,Nom_d_emprunt,Periode,Intervalle_Moyen,Intervalle_Median,Ecart_Type_Intervalle,Longueur_Moyenne,Pourcentage_Annexes,Nombre_Total_Mentions,Pourcentage_Posts_Contenant_Mentions_Pairs,Pourcentage_Posts_Contenant_Mentions_Auteurs,...,Nombre_Total_Hashtags,Nombre_de_Posts,Pourcentage_Posts_Contenant_Hashtags_Total,Plusieurs_Images,Personne,PostProduction,Mots,N_Ecrit,N_Video,N_Audio
0,Agathe,1,23.000000,23.0,NaN,1157.5,0.0,0,0.0,0.0,...,60,2,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
1,Alice,1,1.666667,2.0,0.866025,878.6,0.0,1,10.0,0.0,...,54,10,100.0,0.0,0.0,0.0,0.0,100.0,20.0,0.0
2,Alice,2,7.666667,5.0,6.538348,1322.6,0.0,4,30.0,0.0,...,66,10,100.0,0.0,0.0,0.0,0.0,100.0,20.0,0.0
3,Alice,4,12.444444,12.0,7.763876,1507.3,0.0,2,10.0,0.0,...,68,10,100.0,0.0,0.0,0.0,0.0,100.0,20.0,0.0
4,Aline,1,8.333333,1.0,22.377444,463.9,0.0,4,0.0,10.0,...,0,10,0.0,0.0,10.0,90.0,90.0,100.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Maude,4,19.111111,22.0,9.225568,1087.2,0.0,11,0.0,20.0,...,0,10,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
63,Salomé,1,4.700000,4.0,2.790858,1194.3,0.0,4,0.0,0.0,...,42,10,100.0,10.0,0.0,0.0,0.0,100.0,0.0,0.0
64,Salomé,2,5.777778,4.0,5.517648,1547.6,0.0,11,70.0,0.0,...,118,10,100.0,70.0,10.0,60.0,60.0,100.0,0.0,0.0
65,Salomé,4,4.000000,3.0,3.278719,1659.9,10.0,10,30.0,0.0,...,95,10,100.0,50.0,0.0,0.0,0.0,100.0,0.0,0.0


In [123]:
# Pour obtenir le dataframe nécesaire au clustering sur les enquetés, on va faire des sommes et des moyennes pondérées sur les indicateurs statistiques. 
df = supra_df.copy()

colonnes_a_sommer = [  'Nombre_de_Posts', 'Nombre_Total_Hashtags', 'Nombre_Total_Mentions',
  ]

colonnes_a_pond_moy = [
'Plusieurs_Images',
 'Personne',
 'PostProduction',
 'Mots',
 'N_Ecrit',
 'N_Video',
 'N_Audio',  'Pourcentage_Posts_Contenant_Hashtags_Total', 'Pourcentage_Posts_Contenant_Mentions_Total',  'Longueur_Moyenne',
 'Pourcentage_Annexes',
 'Intervalle_Moyen',
 'Ecart_Type_Intervalle', 
 'Pourcentage_Posts_Contenant_Mentions_Pairs',
 'Pourcentage_Posts_Contenant_Mentions_Auteurs',
 'Pourcentage_Posts_Contenant_Mentions_Maisons',
]

somme = df.groupby("Nom_d_emprunt")[colonnes_a_sommer].sum().reset_index()

for col in colonnes_a_pond_moy:
    df[col + "_pond"] = df[col] * df["Nombre_de_Posts"]

ponds = (
    df.groupby("Nom_d_emprunt")[[col + "_pond" for col in colonnes_a_pond_moy] + ["Nombre_de_Posts"]]
    .sum()
    .reset_index()
)

for col in colonnes_a_pond_moy:
    ponds[col] = ponds[col + "_pond"] / ponds["Nombre_de_Posts"]

ponds = ponds[["Nom_d_emprunt"] + colonnes_a_pond_moy]

df_global = pd.merge(somme, ponds, on="Nom_d_emprunt")

df_global.head()
#df_global.to_excel("Clustering_Global.xlsx", index=False)


,Nom_d_emprunt,Nombre_de_Posts,Nombre_Total_Hashtags,Nombre_Total_Mentions,Plusieurs_Images,Personne,PostProduction,Mots,N_Ecrit,N_Video,N_Audio,Pourcentage_Posts_Contenant_Hashtags_Total,Pourcentage_Posts_Contenant_Mentions_Total,Longueur_Moyenne,Pourcentage_Annexes,Intervalle_Moyen,Ecart_Type_Intervalle,Pourcentage_Posts_Contenant_Mentions_Pairs,Pourcentage_Posts_Contenant_Mentions_Auteurs,Pourcentage_Posts_Contenant_Mentions_Maisons
0,Agathe,2,60,0,0.00,0.00,0.0,0.000,100.000,0.000,0.0,100.000000,0.000000,1157.500000,0.00,23.000000,0.000000,0.000000,0.000,0.000
1,Alice,30,188,7,0.00,0.00,0.0,0.000,100.000,20.000,0.0,100.000000,23.333333,1236.166667,0.00,7.259259,5.056083,16.666667,0.000,0.000
2,Aline,40,423,147,0.00,2.50,67.5,45.000,100.000,0.000,0.0,75.000000,85.000000,1061.975000,0.00,11.250000,15.385258,32.500000,52.500,77.500
3,Alya,30,160,0,100.00,0.00,100.0,100.000,100.000,0.000,0.0,66.666667,0.000000,675.000000,0.00,11.259259,17.537855,0.000000,0.000,0.000
4,Amélie,32,393,72,43.75,18.75,62.5,53.125,96.875,28.125,0.0,68.750000,84.375000,1627.500000,18.75,13.194444,0.708321,0.000000,59.375,34.375
